In [1]:
%load_ext autoreload
%load_ext autotime

time: 318 µs (started: 2023-03-07 21:54:03 -08:00)


In [2]:
%run init.ipynb

/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/core.py:17: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import format_bytes, parse_bytes, tmpfile, get_ip_interface
/usr/workspace/iopp/.conda/envs/jupyter/lib/python3.9/site-packages/dask_jobqueue/htcondor.py:6: FutureWarning: parse_bytes is deprecated and will be removed in a future release. Please use dask.utils.parse_bytes instead.
  from distributed.utils import parse_bytes


{'cls': <class 'distributed.scheduler.Scheduler'>, 'options': {'protocol': 'tcp://', 'interface': None, 'host': None, 'dashboard_address': ':8787', 'security': None}}
time: 3.06 s (started: 2023-03-07 21:54:03 -08:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.31 s (started: 2023-03-07 21:54:06 -08:00)


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.129.185:8787/status,
Dashboard: http://192.168.129.185:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.129.185:32813,Workers: 0
Dashboard: http://192.168.129.185:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 64.7 ms (started: 2023-03-07 21:54:07 -08:00)


In [5]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 793 µs (started: 2023-03-07 21:54:07 -08:00)


In [6]:
DELTA = 0.0001

time: 1.05 ms (started: 2023-03-07 21:54:07 -08:00)


In [7]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-cosmoflow/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=2186,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 1.87 s (started: 2023-03-07 21:54:07 -08:00)


In [8]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,1,0,2099194032,2,0,0.295049,3.254283,MPI_Comm_dup,0,...,app1,,UNKNOWN,0,0,0.0,2.959234,17746659,0,0
1,2,1,0,2099194032,0,3,3.253641,3.253651,open,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000010,32536458,3341432783455051907,-929352282003167793
2,3,1,0,2099194032,0,3,3.253686,3.253690,open,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000004,32536878,3341432783455051907,-929352282003167793
3,4,1,0,2099194032,0,3,3.253701,3.253709,ftruncate,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000008,32537048,3341432783455051907,-929352282003167793
4,5,1,0,2099194032,0,3,3.253717,3.253719,close,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000002,32537177,3341432783455051907,-929352282003167793


time: 50.4 s (started: 2023-03-07 21:54:11 -08:00)


In [ ]:
ddf.index.count().compute()

In [11]:
ddf['tend'].max().compute()

12023159.0

time: 3.57 s (started: 2023-03-07 21:58:05 -08:00)


In [12]:
ddf['file_id'].unique().count().compute()

52779

time: 2.67 s (started: 2023-03-07 21:58:11 -08:00)


In [13]:
ddf['proc_id'].unique().count().compute()

693

time: 2.71 s (started: 2023-03-07 21:58:14 -08:00)


In [14]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
tmid_min, tmid_max

(0, 120231585016530)

time: 3.2 ms (started: 2023-03-07 21:59:53 -08:00)


In [15]:
divmod(tmid_max, 1e7)

(12023158.0, 5016530.0)

time: 2.42 ms (started: 2023-03-07 21:59:53 -08:00)


In [16]:
tmid_max_s = tmid_max / 1e7
tmid_max_s

12023158.501653

time: 2.49 ms (started: 2023-03-07 21:59:53 -08:00)


In [17]:
tmid_max_s / (60 * 60 * 24 * 30)

4.638564236748842

time: 2.71 ms (started: 2023-03-07 21:59:53 -08:00)


In [18]:
60 * 60 * 24 * 30

2592000

time: 2.46 ms (started: 2023-03-07 21:59:53 -08:00)


In [19]:
cm_tmid_max_s = 6683055996 / 1e7
cm_tmid_max_s

668.3055996

time: 2.59 ms (started: 2023-03-07 21:59:54 -08:00)


In [20]:
trange = np.arange(tmid_min, tmid_max, 1e7)
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

time: 3.14 s (started: 2023-03-07 21:59:55 -08:00)


In [21]:
max_io_time = ddf.groupby(['proc_id']).sum()['duration'].max()
max_io_time

dd.Scalar<series-..., dtype=float64>

time: 19.5 ms (started: 2023-03-07 21:59:58 -08:00)


In [22]:
VIEW_TYPES = ['trange', 'file_id', 'proc_id']

time: 651 µs (started: 2023-03-07 21:59:59 -08:00)


In [23]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(ddf: dd.DataFrame, delta: float, max_io_time: float, metric='duration'):
    def set_delta(df: pd.DataFrame):
        df['duration_csp'] = df['duration_sum'].cumsum() / max_io_time
        df['duration_delta'] = df['duration_csp'].diff().fillna(df['duration_csp'])
        df['duration_score'] = np.digitize(df['duration_delta'], bins=DELTA_BINS, right=True)
        df['duration_cut'] = np.choose(df['duration_score'] - 1, choices=DELTA_BINS, mode='clip')
        return df
    ddf = ddf.map_partitions(set_delta)
    return ddf[ddf['duration_delta'] > delta]

time: 1.97 ms (started: 2023-03-07 22:00:00 -08:00)


In [24]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')

main_view = ddf[ddf['cat'] == 0] \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
        'file_name': ['first'],
        'proc_name': ['first']
#         'func_id': [unique_flatten()]
    }) \
    .reset_index() \
    .persist()

main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]

main_view


,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,file_name_first,proc_name_first
npartitions=1,,,,,,,,,,,,
,int64,int64,int64,int64,int64,float32,int64,int64,int64,int64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...


time: 889 ms (started: 2023-03-07 22:00:04 -08:00)


In [25]:
wait(main_view)


DoneAndNotDoneFutures(done={<Future: error, key: ('reset_index-b604e032ef5314731e53a10c9760a174', 0)>}, not_done=set())

time: 2min 1s (started: 2023-03-07 22:00:05 -08:00)


In [ ]:
main_view.tail(2)

In [35]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])


main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
#main_view['metadata_count'] = main_view['metadata_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

main_view['workload_type'] = 0
main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)


final_view = main_view \
    .groupby(['trange', 'file_id', 'proc_id']) \
    .agg({
        'duration_sum': sum,
        'index_count': sum,
        'size_sum': sum,
        'read_count': sum,
        'write_count': sum,
        'metadata_count': sum,
        'read_size': sum,
        'write_size': sum,
        'metadata_size': sum,
        'read_time': sum,
        'write_time': sum,
        'metadata_time': sum,
        'sequential_count': sum,
        'random_count': sum,
        'sequential_size': sum,
        'random_size': sum,
        'sequential_time': sum,
        'random_time': sum,
        'data_count': sum,
        'data_size': sum,
        'data_time': sum,
        'workload_type': sum,
        'size_min': min,
        'size_max': max,
        'file_name_first': 'first',
        'proc_name_first': 'first'
    }) \
    .reset_index() \
    .rename(columns={
        'file_name_first': 'file_name', 
        'proc_name_first': 'proc_name'
    })
mask_hostname = int('0000000000000000111111111111111100000000000000000000000000000000', 2)

final_view['hostname_id'] = final_view['proc_id'] & mask_hostname
final_view = final_view.persist()


time: 402 ms (started: 2023-03-07 15:53:23 -08:00)


In [36]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('assign-e6f5e0d66194bb41e52294d6b4fbf1f7', 0)>}, not_done=set())

time: 3.5 s (started: 2023-03-07 15:53:24 -08:00)


In [37]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,random_time,data_count,data_size,data_time,workload_type,size_min,size_max,file_name,proc_name,hostname_id
0,4,113104469769545699,5117778030848176863,0.000663,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-0,4294967296
1,4,113104469769545699,5117778030848242393,0.000674,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-1,4294967296
2,4,113104469769545699,5117778030848307929,0.000677,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-2,4294967296
3,4,113104469769545699,5117778030848373465,0.000640,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-3,4294967296
4,4,113104469769545699,5117778030848439001,0.000394,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-4,4294967296


time: 36.5 ms (started: 2023-03-07 15:53:29 -08:00)


In [38]:
del main_view

time: 595 µs (started: 2023-03-07 15:53:32 -08:00)


In [39]:
final_view.head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,random_time,data_count,data_size,data_time,workload_type,size_min,size_max,file_name,proc_name,hostname_id
0,4,113104469769545699,5117778030848176863,0.000663,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-0,4294967296
1,4,113104469769545699,5117778030848242393,0.000674,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-1,4294967296
2,4,113104469769545699,5117778030848307929,0.000677,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-2,4294967296
3,4,113104469769545699,5117778030848373465,0.000640,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-3,4294967296
4,4,113104469769545699,5117778030848439001,0.000394,230,0,0,0,230,0,...,0.0,0,0,0.0,1,0,0,UNKNOWN,app1-localhost-4,4294967296


time: 37 ms (started: 2023-03-07 15:53:33 -08:00)


In [40]:
DELTA_BINS = [
    0,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]

def filter_delta(view_type: str, ddf: dd.DataFrame, delta: float, metric: str, max_io_time: dd.core.Scalar):
    metric_col, csp_col, delta_col, score_col, cut_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
        f"{metric}_score",
        f"{metric}_cut"
    )

    def set_delta(df: pd.DataFrame):
        df[csp_col] = df[metric_col].cumsum() / max_io_time
        df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
        df[score_col] = np.digitize(df[delta_col], bins=DELTA_BINS, right=True)
        df[cut_col] = np.choose(df[score_col] - 1, choices=DELTA_BINS, mode='clip')
        return df

    index = pd.MultiIndex(levels=[[1]], codes=[[]], names=[view_type])
    meta = dd.utils.make_meta(([
        (metric_col, np.float64),
        (csp_col, np.float64),
        (delta_col, np.float64),
        (score_col, np.int8),
        (cut_col, np.float16),
    ]), index=index)

#     ddf = ddf.map_partitions(set_delta, meta=meta)
    ddf = ddf.map_partitions(set_delta)

    return ddf[ddf[delta_col] > delta]


time: 1.93 ms (started: 2023-03-07 15:53:34 -08:00)


In [41]:
max_io_time = final_view.groupby(['proc_id']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float64>

time: 13.9 ms (started: 2023-03-07 15:53:35 -08:00)


In [42]:
import dask

dask.__version__


'2021.08.1'

time: 2.36 ms (started: 2023-03-07 15:53:36 -08:00)


In [43]:
import itertools as it

def _set_metric_percentages(ddf: dd.DataFrame, max_io_time: dd.core.Scalar, metric: str):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    ddf[pero_col] = ddf[metric_col] / max_io_time
    ddf[perr_col] = ddf[metric_col] / ddf[metric_col].sum()
    return ddf

def _compute_expanded_view(
    parent_view: dd.DataFrame,
    view_type: str,
    cols: tuple,
    metric: str,
    delta: float,
    max_io_time: dd.core.Scalar
):
    # Get column names
    metric_col, score_col, cut_col = cols
    # Check view type
    if view_type is not 'proc_id':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_id']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Filter view
    filtered_view = filter_delta(
        view_type=view_type,
        ddf=group_view,
        delta=delta,
        metric=metric,
        max_io_time=max_io_time
    )
    # Find filtered records and set duration scores
#     expanded_view = parent_view \
#         .query(f"{view_type}.isin(@indices)", local_dict={'indices': filtered_view.index.unique()}) \
#         .drop(columns=[score_col, cut_col], errors='ignore') \
#         .merge(score_view, on=[view_type])
    expanded_view = parent_view \
        .query(f"{view_type} in @indices", local_dict={'indices': filtered_view.index.unique()}) 
    
    expanded_view = filter_delta(
        view_type=view_type,
        ddf=expanded_view,
        delta=0,
        metric=metric,
        max_io_time=max_io_time
    )
    
    # trange, proc_id, file_id
    # trange
    # - proc_id
    # -- file_id --> duration_score
    
    # Set metric percentages
    expanded_view = _set_metric_percentages(ddf=expanded_view, max_io_time=max_io_time, metric=metric)
    # Return expanded view
    return expanded_view

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type]['expanded_view'] if parent_type in views else final_view
    # Create colum names
    metric_col, score_col, cut_col = f"{metric}_sum", f"{metric}_score", f"{metric}_cut"
    # Compute expanded view
    expanded_view = _compute_expanded_view(
        parent_view=parent_view,
        view_type=view_type,
        cols=(metric_col, score_col, cut_col),
        metric=metric,
        delta=0.0001,
        max_io_time=max_io_time
    )
    # Return views
    views[view_permutation] = {}
    views[view_permutation]['expanded_view'] = expanded_view
    views[view_permutation]['bottleneck_view'] = expanded_view[expanded_view[cut_col] >= 0.5]

len(views)

15

time: 2.38 s (started: 2023-03-07 15:53:36 -08:00)


In [44]:
views[('trange',)]['expanded_view'].head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,workload_type,size_min,size_max,file_name,proc_name,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
0,5,1993729102949669912,5117778030848157135,0.000032,2,0,0,0,2,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-0,4294967296,1,0.0,2.416918e-07,2.659850e-09
1,5,1993729102949669912,5117778030848222671,0.001372,62,0,0,0,62,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-1,4294967296,1,0.0,1.023833e-05,1.126742e-07
2,5,1993729102949669912,5117778030848288207,0.000033,2,0,0,0,2,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-2,4294967296,1,0.0,2.454216e-07,2.700898e-09
3,5,1993729102949669912,5117778030848353743,0.000033,2,0,0,0,2,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-3,4294967296,1,0.0,2.469135e-07,2.717317e-09
4,5,1993729102949669912,5117778030848484815,0.000680,28,0,0,0,28,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-5,4294967296,1,0.0,5.074782e-06,5.584865e-08


time: 1.63 s (started: 2023-03-07 15:53:38 -08:00)


In [45]:
views[('file_id',)]['expanded_view'].head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,workload_type,size_min,size_max,file_name,proc_name,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
0,4,1993729102949669912,5117778030848157135,0.000062,4,0,0,0,4,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-0,4294967296,1,0.0,4.632426e-07,5.069959e-09
1,4,1993729102949669912,5117778030848222671,0.000132,8,0,0,0,8,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-1,4294967296,1,0.0,9.854162e-07,1.078489e-08
2,4,1993729102949669912,5117778030848288207,0.000063,4,0,0,0,4,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-2,4294967296,1,0.0,4.699563e-07,5.143436e-09
3,4,1993729102949669912,5117778030848353743,0.000069,4,0,0,0,4,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-3,4294967296,1,0.0,5.147140e-07,5.633288e-09
4,4,1993729102949669912,5117778030848419279,0.000067,4,0,0,0,4,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-4,4294967296,1,0.0,4.990488e-07,5.461840e-09


time: 914 ms (started: 2023-03-07 15:53:40 -08:00)


In [46]:
views[('trange','file_id')]['expanded_view'].head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,workload_type,size_min,size_max,file_name,proc_name,hostname_id,duration_pero,duration_perr,duration_score,duration_cut
0,5,1993729102949669912,5117778030848157135,0.000032,2,0,0,0,2,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-0,4294967296,2.416918e-07,2.660831e-09,1,0.0
1,5,1993729102949669912,5117778030848222671,0.001372,62,0,0,0,62,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-1,4294967296,1.023833e-05,1.127158e-07,1,0.0
2,5,1993729102949669912,5117778030848288207,0.000033,2,0,0,0,2,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-2,4294967296,2.454216e-07,2.701893e-09,1,0.0
3,5,1993729102949669912,5117778030848353743,0.000033,2,0,0,0,2,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-3,4294967296,2.469135e-07,2.718318e-09,1,0.0
4,5,1993729102949669912,5117778030848484815,0.000680,28,0,0,0,28,0,...,1,0,0,/sys/devices/system/node/node254/meminfo,app1-localhost-5,4294967296,5.074782e-06,5.586924e-08,1,0.0


time: 1.49 s (started: 2023-03-07 15:53:41 -08:00)


In [47]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_id=('file_name', 'proc_name', 'trange'),
    proc_id=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_id='file_name',
    proc_id='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    ll_view: pd.DataFrame,
    ml_view: pd.DataFrame,
    hl_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = ll_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = hl_view.loc[hl_id]
        ml_row = ml_view.loc[(hl_id, ml_id)]
        ll_row = ll_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view_dict in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view_dict=view_dict,
            view_types=view_types,
        )
    # Generate bottlenecks
#     bottlenecks = _process_bottleneck_views(bottleneck_views=bottleneck_views)
    # Return bottleneck views
    return bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, pd.DataFrame]):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        bottlenecks_delayed.append(_process_bottleneck_view(
            view_key=view_key,
            ll_view=view_dict['low_level_view'],
            ml_view=view_dict['mid_level_view'],
            hl_view=view_dict['high_level_view']
        ))
    # Compute all bottlenecks
    futures = compute(*bottlenecks_delayed, sync=False)
    results = get_client().gather(list(futures))
    for view_key, result in results:
        bottlenecks[view_key] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
        view_key: tuple,
        view_dict: Dict[str, dd.DataFrame],
        view_types: list
    ):
        # Get view type
        view_type = view_key[-1]
        bottleneck_type = BOTTLENECK_TYPE[view_type]

        # Get parent view
        bottleneck_view = view_dict['bottleneck_view']

        # Create lower level view
        low_level_view = bottleneck_view \
            .groupby(list(BOTTLENECK_ORDER[view_type])) \
            .first() \
            .drop(columns=['acc_pat', 'io_cat'], errors='ignore')

        # Non-proc agg columns
        non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
        proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

        if bottleneck_type is not 'proc_name':

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'proc_name']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(proc_agg_dict)

        else:

            mid_level_view = low_level_view \
                .reset_index() \
                .groupby([bottleneck_type, 'trange']) \
                .agg(non_proc_agg_dict)

            high_level_view = mid_level_view \
                .reset_index() \
                .groupby([bottleneck_type]) \
                .agg(non_proc_agg_dict)

        return dict(
            low_level_view=low_level_view.persist(),
            mid_level_view=mid_level_view.persist(),
            high_level_view=high_level_view.persist()
        )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    agg_dict['duration_cut'] = 'first'
    agg_dict['duration_score'] = 'first'
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 7.65 ms (started: 2023-03-07 15:53:46 -08:00)


In [48]:
select_view = ('proc_id',)

views2 = {}
views2[select_view] = views[select_view]

views2

{('proc_id',): {'expanded_view': Dask DataFrame Structure:
                trange file_id proc_id duration_sum index_count size_sum read_count write_count metadata_count read_size write_size metadata_size read_time write_time metadata_time sequential_count random_count sequential_size random_size sequential_time random_time data_count data_size data_time workload_type size_min size_max file_name proc_name hostname_id duration_score duration_cut duration_pero duration_perr
  npartitions=1                                                                                                                                                                                                                                                                                                                                                                                                                  
                 int64   int64   int64      float32       int64    int64      int64       int64          

time: 41 ms (started: 2023-03-07 15:53:48 -08:00)


In [49]:
len(views[select_view]['bottleneck_view'])

690079

time: 1.08 s (started: 2023-03-07 15:53:48 -08:00)


In [50]:
views[select_view]['bottleneck_view'].head()

,trange,file_id,proc_id,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,...,workload_type,size_min,size_max,file_name,proc_name,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
259872,650,113104469769545699,5117778030848373656,0.000084,35,0,0,0,35,0,...,1,0,0,UNKNOWN,app1-localhost-3,4294967296,5,0.5,6.258624e-07,6.845929e-09
259873,650,3316303174978039817,5117778030848373656,0.000008,1,0,0,0,1,0,...,1,0,0,/proc/106364/task,app1-localhost-3,4294967296,5,0.5,5.893103e-08,6.446107e-10
259874,101772,113104469769545699,5117778030848373656,0.000005,2,0,0,0,2,0,...,1,0,0,UNKNOWN,app1-localhost-3,4294967296,5,0.5,3.879001e-08,4.243004e-10
259875,101772,3341432782168351319,5117778030848373656,0.000031,4,0,0,0,4,0,...,1,0,0,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,app1-localhost-3,4294967296,5,0.5,2.312484e-07,2.529485e-09
259876,101772,3341432782742984937,5117778030848373656,0.000029,5,0,0,0,5,0,...,1,0,0,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,app1-localhost-3,4294967296,5,0.5,2.155831e-07,2.358133e-09


time: 1.13 s (started: 2023-03-07 15:53:49 -08:00)


In [51]:
bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottleneck_views)

15

time: 3.77 s (started: 2023-03-07 15:53:50 -08:00)


In [59]:
for view_key, bottleneck_view_dict in bottleneck_views.items():
    print(view_key, 'hl_view', len(bottleneck_view_dict['high_level_view']), 'ml_view', len(bottleneck_view_dict['mid_level_view']), 'll_view', len(bottleneck_view_dict['low_level_view']))

('trange',) hl_view 0 ml_view 0 ll_view 0
('file_id',) hl_view 0 ml_view 0 ll_view 0
('proc_id',) hl_view 120 ml_view 287384 ll_view 690079
('trange', 'file_id') hl_view 0 ml_view 0 ll_view 0
('trange', 'proc_id') hl_view 120 ml_view 211258 ll_view 493025
('file_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('file_id', 'proc_id') hl_view 120 ml_view 286750 ll_view 502467
('proc_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'file_id') hl_view 0 ml_view 0 ll_view 0
('trange', 'file_id', 'proc_id') hl_view 120 ml_view 211215 ll_view 364858
('trange', 'proc_id', 'file_id') hl_view 0 ml_view 0 ll_view 0
('file_id', 'trange', 'proc_id') hl_view 120 ml_view 211222 ll_view 364865
('file_id', 'proc_id', 'trange') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'trange', 'file_id') hl_view 0 ml_view 0 ll_view 0
('proc_id', 'file_id', 'trange') hl_view 0 ml_view 0 ll_view 0
time: 3.91 s (started: 2023-03-07 15:56:48 -08:00)


In [56]:
bottleneck_views[('trange',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,data_size,data_time,workload_type,size_min,size_max,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
trange,,,,,,,,,,,,,,,,,,,,,


time: 18.6 ms (started: 2023-03-07 15:54:35 -08:00)


In [54]:
bottleneck_views[('file_id',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,data_size,data_time,workload_type,size_min,size_max,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
file_name,,,,,,,,,,,,,,,,,,,,,


time: 19.4 ms (started: 2023-03-07 15:54:04 -08:00)


In [55]:
bottleneck_views[('proc_id',)]['high_level_view'].compute()

,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,data_size,data_time,workload_type,size_min,size_max,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
proc_name,,,,,,,,,,,,,,,,,,,,,
app1-localhost-0,134.055023,39556,13023841792,21729,0,17827,13023841792,0,0,132.262238,...,13023841792,132.262238,7344,0,2097152,22728966930432,7,1.00,1.000000,0.010938
app1-localhost-1,98.030266,33576,13022881664,15520,0,18056,13022881664,0,0,97.513901,...,13022881664,97.513901,8350,0,2097152,27569395073024,5,0.50,0.731269,0.007999
app1-localhost-10,97.698807,29311,13022881664,15520,0,13791,13022881664,0,0,97.261122,...,13022881664,97.261122,7867,0,2097152,24550033063936,5,0.50,0.728796,0.007972
app1-localhost-100,115.398628,46787,13023841792,21729,0,25058,13023841792,0,0,113.905459,...,13023841792,113.905459,9448,0,2097152,30657476558848,6,0.75,0.860830,0.009416
app1-localhost-101,97.735054,27696,13022881664,15520,0,12176,13022881664,0,0,97.209074,...,13022881664,97.209074,7410,0,2097152,22209275887616,5,0.50,0.729067,0.007975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
app1-localhost-95,97.855499,27984,13022881664,15520,0,12464,13022881664,0,0,97.382728,...,13022881664,97.382728,7584,0,2097152,23042499543040,5,0.50,0.729965,0.007985
app1-localhost-96,113.724113,46680,13023841792,21729,0,24951,13023841792,0,0,112.243841,...,13023841792,112.243841,9330,0,2097152,30468497997824,6,0.75,0.848339,0.009279
app1-localhost-97,98.202629,28071,13022881664,15520,0,12551,13022881664,0,0,97.713466,...,13022881664,97.713466,7412,0,2097152,22436909154304,5,0.50,0.732555,0.008013


time: 35.5 ms (started: 2023-03-07 15:54:06 -08:00)


In [62]:
bottleneck_views[('proc_id',)]['mid_level_view'].compute()

duration_sum  index_count  size_sum  read_count  \
proc_name         trange                                                     
app1-localhost-0  651          0.000076           36         0           0   
                  107353       0.084053           33   8395744          15   
                  110365       0.083528           31   8391648          14   
                  121985       0.086115           21   8391648          14   
                  123703       0.000085            9         0           0   
...                                 ...          ...       ...         ...   
app1-localhost-99 3383107      0.061415           24   8391032          10   
                  3384825      0.048974           18   2099048           5   
                  3384826      0.001244            6   6291984           5   
                  3386544      0.067138           23   8391032          10   
                  3388262      0.068136           24   8391032          10   

                           write_count  metadata_count  read_size  write_size  \
proc_name         trange                                                        
app1-localhost-0  651                0              36          0           0   
                  107353             0              18    8395744           0   
                  110365             0              17    8391648           0   
                  121985             0               7    8391648           0   
                  123703             0               9          0           0   
...                                ...             ...        ...         ...   
app1-localhost-99 3383107            0              14    8391032           0   
                  3384825            0              13    2099048           0   
                  3384826            0               1    6291984           0   
                  3386544            0              13    8391032           0   
                  3388262            0              14    8391032           0   

                           metadata_size  read_time  ...  data_size  \
proc_name         trange                             ...              
app1-localhost-0  651                  0   0.000000  ...          0   
                  107353               0   0.083709  ...    8395744   
                  110365               0   0.064725  ...    8391648   
                  121985               0   0.085754  ...    8391648   
                  123703               0   0.000000  ...          0   
...                                  ...        ...  ...        ...   
app1-localhost-99 3383107              0   0.060982  ...    8391032   
                  3384825              0   0.048708  ...    2099048   
                  3384826              0   0.001174  ...    6291984   
                  3386544              0   0.066633  ...    8391032   
                  3388262              0   0.067722  ...    8391032   

                           data_time  workload_type  size_min  size_max  \
proc_name         trange                                                  
app1-localhost-0  651       0.000000              2         0         0   
                  107353    0.083709              8         0   2097152   
                  110365    0.064725              5         0   2097152   
                  121985    0.085754              3         0   2097152   
                  123703    0.000000              2         0         0   
...                              ...            ...       ...       ...   
app1-localhost-99 3383107   0.060982              5         0   2097152   
                  3384825   0.048708              5         0   2097152   
                  3384826   0.001174              2         0   2097152   
                  3386544   0.066633              5         0   2097152   
                  3388262   0.067722              5         0   2097152   

                           hostname_id  duration_score  du

time: 335 ms (started: 2023-03-07 16:05:09 -08:00)


In [69]:
proc_id_bview = bottleneck_views[('proc_id',)]['low_level_view'].compute()
proc_id_bview = proc_id_bview.drop(columns=['file_id', 'proc_id'])
proc_id_bview

duration_sum  \
proc_name         trange  file_name                                                          
app1-localhost-0  651     /proc/106360/task                                       0.000008   
                          UNKNOWN                                                 0.000068   
                  107353  /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...      0.000040   
                          /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...      0.000023   
                          /etc/romio-hints                                        0.000005   
...                                                                                    ...   
app1-localhost-99 3386544 UNKNOWN                                                 0.000003   
                  3388262 /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...      0.000036   
                          /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...      0.000062   
                          /p/gpfs1/brainusr/datasets/cosmoflow/cosmoUnive...      0.068034   
                          UNKNOWN                                                 0.000004   

                                                                              index_count  \
proc_name         trange  file_name                                                         
app1-localhost-0  651     /proc/106360/task                                             1   
                          UNKNOWN                                                      35   
                  107353  /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...            5   
                          /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...            4   
                          /etc/romio-hints                                              2   
...                                                                                   ...   
app1-localhost-99 3386544 UNKNOWN                                                       2   
                  3388262 /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...            5   
                          /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...            5   
                          /p/gpfs1/brainusr/datasets/cosmoflow/cosmoUnive...           12   
                          UNKNOWN                                                       2   

                                                                              size_sum  \
proc_name         trange  file_name                                                      
app1-localhost-0  651     /proc/106360/task                                          0   
                          UNKNOWN                                                    0   
                  107353  /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...         0   
                          /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...         0   
                          /etc/romio-hints                                        4096   
...                                                                                ...   
app1-localhost-99 3386544 UNKNOWN                                                    0   
                  3388262 /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...         0   
                          /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...         0   
                          /p/gpfs1/brainusr/datasets/cosmoflow/cosmoUnive...   8391032   
                          UNKNOWN                                                    0   

                                                                              read_count  \
proc_name         trange  file_name                                                        
app1-localhost-0  651     /proc/106360/task                                            0   
                          UNKNOWN                                                      0   
                  107353  /dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-co...           0   
              

time: 1.22 s (started: 2023-03-07 16:08:32 -08:00)


In [73]:
proc_id_bview.loc['app1-localhost-0'].query('duration_perr > 0.00001').sort_values('duration_sum', ascending=False)

,,duration_sum,index_count,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,...,data_size,data_time,workload_type,size_min,size_max,hostname_id,duration_score,duration_cut,duration_pero,duration_perr
trange,file_name,,,,,,,,,,,,,,,,,,,,,
2052321,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a10415786_2_0_3_float.hdf5,2.339140,1,2097152,1,0,0,2097152,0,0,2.339140,...,2097152,2.339140,1,2097152,2097152,4294967296,7,1.0,0.017449,0.000191
1833259,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a1059893_0_1_1_float.hdf5,0.409448,19,8391648,14,0,5,8391648,0,0,0.409110,...,8391648,0.409110,2,0,2097152,4294967296,7,1.0,0.003054,0.000033
1380959,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a10335103_0_1_2_float.hdf5,0.218689,15,6293968,11,0,4,6293968,0,0,0.218468,...,6293968,0.218468,2,0,2097152,4294967296,7,1.0,0.001631,0.000018
395604,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a10251871_3_2_3_float.hdf5,0.180797,19,8391648,14,0,5,8391648,0,0,0.180508,...,8391648,0.180508,2,0,2097152,4294967296,7,1.0,0.001349,0.000015
2571214,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a10850998_3_1_0_float.hdf5,0.165343,19,8391648,14,0,5,8391648,0,0,0.165131,...,8391648,0.165131,2,0,2097152,4294967296,7,1.0,0.001233,0.000013
2593979,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a10155508_1_1_0_float.hdf5,0.155691,19,8391648,14,0,5,8391648,0,0,0.155367,...,8391648,0.155367,2,0,2097152,4294967296,7,1.0,0.001161,0.000013
4269579,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a8882343_2_0_0_float.hdf5,0.149204,19,8391648,14,0,5,8391648,0,0,0.070571,...,8391648,0.070571,2,0,2097152,4294967296,7,1.0,0.001113,0.000012
4118809,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a8832540_2_0_0_float.hdf5,0.146769,19,8391648,14,0,5,8391648,0,0,0.108608,...,8391648,0.108608,2,0,2097152,4294967296,7,1.0,0.001095,0.000012
663638,/p/gpfs1/brainusr/datasets/cosmoflow/cosmoUniverse_2019_05_4parE/hdf5_transposed_dim128_float/all/univ_ics_2019-03_a10620858_0_3_1_float.hdf5,0.146277,19,8391648,14,0,5,8391648,0,0,0.145930,...,8391648,0.145930,2,0,2097152,4294967296,7,1.0,0.001091,0.000012


time: 49.5 ms (started: 2023-03-07 16:16:54 -08:00)
